<img src="images/GEMS Informatics Learning.png" width=600 alt="GEMS Learning Logo" title="GEMS Learning" />

# **Geostatistics and Interpolation in R**

**Instructors:**  
Ali Joglekar, University of Minnesota (joglekar@umn.edu)  
Yuan Chai, University of Minnesota (chaix026@umn.edu)  

---

## Method of Delivery
- Workshop text, images and R code are all contained within a Jupyter Notebook hosted on the GEMS Informatics Platform. You do not need to have R or RStudio installed on your machine to participate
- All needed data and scripts are in the following github repo: [https://github.com/abjoglekar/GEMS-X003-Geostatistics-Interpolation-R](https://github.com/abjoglekar/GEMS-X003-Geostatistics-Interpolation-R.git)
- A recording of the workshop will be posted on Canvas
- Please download any desired materials, as we cannot guarantee access to the Canvas course beyond 3 months.  

## Module Outline
- [Introduction to geostatistics](#Geostat)  
- [Interpolation target area](#Target)
    - [Exercise 1](#Ex1)
- [Weighted average principal](#Wgts)  
    - [Exercise 2](#Ex2)
- [Kriging interpolation](#Kriging)  
    - [Exercise 3](#Ex3)
- [Automatic variogram fitting](#Autofit)  
    - [Exercise 4](#Ex4)
  

## Setup

### Fetch Script & Data
1. Login to GEMS Platform at [https://gems.agroinformatics.org/](https://gems.agroinformatics.org/)
   GEMS Platform uses Globus to authenticate your account, so if your institution is already linked to Globus (for example, University of Minnesota and many other universities), you can search and select your institution from the list and use your institutional account to log into GEMS Platform. Alternatively, you can log in using Google or ORCID iD, or create your own Globus account to log in.
   
2. Once logged in, click `Analyze > JupyterLab` from the homepage

3. Open a bash terminal by clicking ‘Terminal’ icon in the Launcher OR by clicking `File > New > Terminal` 

4. In bash terminal, create directories for this workshop

    ```bash
    mkdir classes   #if you don't already have this directory created
    cd classes  
    mkdir GEMSX003  #if you don't already have this directory created
    cd GEMSX003
    ```


5. Clone repository for this class

    ```bash
    git clone https://github.com/abjoglekar/GEMS-X003-Geostatistics-Interpolation-R.git
    ```
  
  
### Load R Libraries

In [1]:
# Install and load packages needed for this workshop (

packages_to_load <- c("gstat", "sf", "tmap", "stars", "sp", "lattice", "automap")

for ( package in packages_to_load ) {
    # Check if package is already installed, if not, install and load the package
    if (!require(package, character.only=T, quietly=T, warn.conflicts=F)) {
        install.packages(package)
        suppressPackageStartupMessages(library(package, character.only=T, quietly=T, warn.conflicts=F))
    }
}

Installing package into ‘/home/u01000010/r_libs’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sf’, ‘sftime’


Warning message in install.packages(package):
“installation of package ‘sf’ had non-zero exit status”
Warning message in install.packages(package):
“installation of package ‘sftime’ had non-zero exit status”
Warning message in install.packages(package):
“installation of package ‘gstat’ had non-zero exit status”


ERROR: Error in library(package, character.only = T, quietly = T, warn.conflicts = F): there is no package called ‘gstat’


<details>
    
<summary><span style='color:Green'> Note: For your own work, once packages are installed, it is recommended to just use library() to load your packages  </span></summary>


```r
# General dataframe libraries
library(rio)

# Spatial data libraries
library(sf)
library(spData)

# Plotting libraries
library(tmap)
```
          
</details>


If you run into errors trying to install any packages, please refer to the [R troubleshooting.pdf](https://canvas.umn.edu/courses/343084/files/31161241?module_item_id=8966858) guide on Canvas.

### Data

Your exercises will draw on two vector datasets from the Minnesota Geospatial Commons, which have been downloaded, cleaned, transformed and saved in the directory `./data/MN` for this workshop.
1. \[POINT\] Minnesota Soil, Till, and Ground-Water Geochemical Data: https://conservancy.umn.edu/handle/11299/117364
2. \[RASTER\] Minnesota Digital Elevation Model - 30 Meter Resolution: https://gisdata.mn.gov/dataset/elev-30m-digital-elevation-model


### Attribution

These slides build off of material presented in [Schanbenberger & Pierce 2002](https://www.routledge.com/Contemporary-Statistical-Models-for-the-Plant-and-Soil-Sciences/Schabenberger-Pierce/p/book/9781584881117), [Bivand et al. 2013](https://link.springer.com/book/10.1007/978-1-4614-7618-4), [Pebesma & Bivand 2022](https://r-spatial.org/book/), and [Michael Dorman 2022](http://132.72.155.230:3838/r/index.html).

---

<a id="Geostat"></a>
## Geostatistics Introduction

**Geostatistics** is the statistical methodology for spatial interpolation, concerned with 
modeling, prediction and simulation of spatially continuous phenomena. Where **spatial interpolation**
is the process of estimating values of spatially continuous based on a limited number of observation locations

- Pattern of observation locations not of primary interest
- Interest in inference of aspects of the variable that have not been measured
- Characterize spatial and temporal phenomena (e.g., weather data)
- Geostatistics are used to circumvent data scarcity

Spatial interpolation model: a set of procedures to calculate predicted values of 
the variable of interest, given calibration data. Calibration data usually include:

- **Field measurements**: available for a limited number of locations (e.g., rainfall 
data from meteorological stations)
- **Covariates**: available for each and every location within the area of interest 
(e.g., elevation data from a DEM)

Two categories of spatial interpolation models:

- **Deterministic models**: models using arbitrary parameter values (e.g., IDW)
- **Statistical models**: models using parameters chosen objectively based on the data (e.g., Kriging)

The field of geostatistics was pioneered by Danie Krige and Georges Matheron

- Daniel G. Krige: South African mining engineer who first used GLS prediction with spatial covariances in the 1950's. 
- Georges Matheron: coined this method **kriging** in the 1960's, although similar approaches used previously in meteorology.

### Random Experiment Realizations

Geostatistics deals with the analysis of random fields $Z(s)$

- $Z$ is the attribute of interest being measured
- $s$ is the location at which we observe this attribute
- $Z(s)$ is a spatial observation of attribute $Z$ at location $s$
- $s$ is a point in ${\rm I\!R}^2$, two-dimensional Euclidean space 

Typically, measurements on $Z$ are available a limited number of sometimes arbitrarily 
chosen sample locations, and prediction (*spatial interpolation*) of $Z$ is required 
at non-observed locations $s_0$

A set of spatial data is considered a realization of a random experiment. For any 
outcome $\omega$ of the experiment, a single realization of $Z(s)$ is obtained. 

- $Z(s_0)$ is a random variable by considering the distribution of all possible 
realizations at the location $s_0$
- When a random field is sampled, samples are drawn from one particular realization 
of the random experiment

Consider four realizations of a random function in ${\rm I\!R}^2$ across a continuous 
domain $D$. Realizations at $s_0=[2,3]$ are shown as dots. 

<img src="images/four_random_fields.png" width=600 alt="Random Fields" title="Four Random Fields" />

Imagine a soil sample is poured from a bucket onto a flat surface and the depth of 
the soil is measured. Every pouring $\omega$ constitutes a random experiment, the 
result is a function $Z( \bullet ,\omega)$

Whether we sample the spatial attribute by randomly placing sample locations in $D$ or with a systematic grid has no bearing on the randomness of $Z(s)$

- A spatial attribute is not considered random because we performed random sampling
- Even if we observed $Z(s)$ everywhere on $D$ with an exhaustive sampling procedure, we would be assessing only one realization of the random function $Z(s,\omega)$
- The sample is drawn from one panel of the figure above – a sample $Z(s_1),…,Z(s_n)$ is a sample size of one

### Example: Zinc Concentration

Soil monitoring for zinc along flood plain of the river Meuse, near the village Stein in 
the Netherlands

- The data consist of 155 samples, $Z(s_{1}),…,Z(s_{155})$, where $Z(s_i)$ denotes the zinc level at location $s_i$
- Observations likely not independent as random sample would imply

In [ ]:
library(sf)
library(sp)
library(tmap)

data("meuse")
meuse.sf <- st_as_sf(meuse, coords = c("x", "y"), crs = 28992)

tm_shape(meuse.sf) + tm_bubbles("zinc")

Zinc concentration is larger closer to the river Meuse banks

### Explore Data

Let's look at the spatial relationship more explicitly by plotting zinc as function of distance to river:

In [ ]:
library(lattice)
xyplot(log(zinc) ~ sqrt(dist), meuse)

Strong spatial trend in zinc, and we will want to model that spatial realization over a target area

<a id="Target"></a>
## Interpolation Target Area

When we interpolate a variable, we have to decide *where* we want to interpolate

- Typically done on a regular grid covering area of interest
- Create a `stars` object with a raster covering the target area, and NA’s outside it

The `sp::meuse` data is accompanied by a `sp::meuse.grid` dataset

- Convert the `sp` object into an `sf POINT` object 
- Create a raster `stars` object using `stars::st_rasterize` 

In [ ]:
data("meuse.grid")
meuse.gridSF <- st_as_sf(meuse.grid, coords = c("x","y"), crs = st_crs(meuse.sf))
meuse.gridStars <- stars::st_rasterize(meuse.gridSF)
meuse.gridStars <- st_crop(meuse.gridStars, meuse.gridSF)

In [ ]:
tm_shape(meuse.gridStars) + tm_raster("soil") + 
  tm_layout(main.title = "Gridded Area of Interest for Interpolation")

<a id="Ex1"></a>
### **<span style='color:Green'> Exercise 1: Interpolation Target Area (20 minutes)</span>**   

Your exercises draw on two datasets:

- POINT: [Minnesota Soil, Till, and Ground-Water Geochemical Data](https://conservancy.umn.edu/handle/11299/117364)
- RASTER: [Minnesota Digital Elevation Model - 30 Meter Resolution](https://gisdata.mn.gov/dataset/elev-30m-digital-elevation-model)

1. Import the following shapefiles as `sf` objects

    - Well water sample geochemical data: `water_data_plots` [name this object `h2o_geochem`]
    
2. Import the following geoTiff files as `RasterLayer` objects

    - Digital elevation model: `digital_elevation_mo_1` [name this object `dem`]
    
3. Download a county-level shapefile for Minnesota. 

    1. Using the `raster::getData()` function, download the GADM level 2 (i.e., district) 
    boundary data for the United States. Within your `getData` call
    save this data in a new folder within your "./data" called "bdry_state". Name this 
    object `us_adm2`. [Occassionally, the GADM server is down, try again later or ask me 
    for the shapefile.]
    
    2. Subset `us_adm2` to the county boundaries (level 2) for the state of Minnesota (level 1)
    [name this object `mn_adm2`]. Transform it to an `sf` object.

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
library(sf)
h2o_geochem <- st_read("./data/geochem")

library(raster)
dem <- raster("./data/dem_30m/digital_elevation_mo_1.tif")

library(sp)
us_adm2 <- getData(name = "GADM", country = "USA", level = 2, 
                   download = TRUE, path = "./data/bdry_state")

mn_adm2 <- st_as_sf(us_adm2[us_adm2$NAME_1 == "Minnesota",])
```
----
          
</details>

4. Check that all of your geospatial data files have matching NAD83 / UTM zone 15N 
coordinate reference systems. Correct if necessary. 

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
st_crs(h2o_geochem) == st_crs(mn_adm2)  #does not match
mn_adm2 <- st_transform(mn_adm2, st_crs(h2o_geochem))
```
----
          
</details>

5. The term Iron Range refers to a number of elongated iron-ore mining districts 
around Lake Superior in the United States and Canada. From a geological perspective, 
Minnesota's Iron Range includes these four major iron deposits:

    - Mesabi Range, the largest iron range, largely within Itasca and Saint Louis counties;
    - Vermilion Range, northeast of the Mesabi, in Saint Louis and Lake counties;
    - Gunflint Range, in the extreme northern portion of Cook County and extending into Canada; and
    - Cuyuna Range, southwest of the Mesabi, largely within Crow Wing County.
    
    The remainder of section will focus on estimating the iron levels in the groundwater throughout 
    the largest iron range in Minnesota -- the Mesabi Range -- based on the `h2o_geochem` samples. 
    First, you'll need to prep your data layers. 
    
    1. The MN counties considered part of the Mesabi Range are Itasca and Saint Louis. Subset 
    your `mn_adm2` object to these two counties [name this object `mesabiRngCnty`].
    2. Extract the well water samples taken within the Mesabi Range [name this object `mesabiRngWells`]. 
    3. Crop and mask your `dem90` layer using your `mesabiRngCnty` object [name this object `mesabiRngDEM`]. 
    4. Aggregate your `mesabiRngDEM` layer by a factor of ten (10), assigning the average to the new cells.
    5. For Universal Kriging, any covariates in your calibration data also need to be availabe for every 
    location within your area of interest. Using your `mesabiRngDEM` object, add an elevation covariate to 
    your `mesabiRngWells` object. 
    6. Convert your `mesabiRngDEM` RasterLayer object into a `stars` object using the `stars::st_as_stars()`
    function so that you have a target interpolation layer.

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
library(tmap)
    
cntyInt <- c('Itasca', 'Saint Louis')
mesabiRngCnty <- mn_adm2[mn_adm2$NAME_2 %in% cntyInt,]

lakeSup <- mn_adm2[mn_adm2$NAME_2 == "Lake Superior",]

cntyMap <- tm_shape(mn_adm2) + tm_borders() +
  tm_shape(mesabiRngCnty) + tm_polygons("red") +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_layout(main.title = "Map of Minnesota with the Mesabi Range \n Counties Highlighted", 
            main.title.size = 1)

mesabiRngWells <- h2o_geochem[mesabiRngCnty,]

mesabiRngDEM <- mask(crop(dem, mesabiRngCnty), mesabiRngCnty)

mesabiRngDEM <- aggregate(mesabiRngDEM, fact = 10, fun=mean)
names(mesabiRngDEM) <- "elevation"

wellsDEM <- extract(mesabiRngDEM, mesabiRngWells)
mesabiRngWells$elevation <- wellsDEM 

library(stars)
mesabiRngDEM <- st_as_stars(mesabiRngDEM)

IRMap <- tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(mesabiRngDEM) + tm_raster(title = "Elevation", style = "fisher") +
  tm_shape(mesabiRngWells) + tm_bubbles("Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "Well Water Samples in the Mesabi Range", 
            main.title.size = 1, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

tmap_arrange(cntyMap, IRMap, nrow = 1)
```
----
          
</details>

<a id="Wgts"></a>
## Weighted Average Principle

Many of the commonly used interpolation methods -- including today's focus: IDW & Kriging -- assume 
a predicted value is a *weighted average* of neighboring points. 

$$
\hat Z(s_0) = \frac{\sum_{i=1}^{n}w(s_i)Z(s_i)}{\sum_{i-1}^{n}w(s_i)}
$$
where

- $\hat Z(s_0)$ is the predicted value at location $s_0$
- $w(s_i)$ is the weight of measured point $i$
- $Z(s_i)$ is the value of measured point $i$

### IDW Interpolation Weights

IDW weights
$$
\hat Z(s_0) = \frac{\sum_{i=1}^{n}w_iZ(s_i)}{\sum_{i-1}^{n}w_i}
$$
with $w_i = |s_0 - s_i|^{-p}$, where $p$, the inverse distance power, defaults to $p=2$

$p$ determines how steeply weight increases with proximity

- Low $p$: weights are more or less equally distributed among neighbors [-> average of all measured points]
- High $p$: one point (the nearest) has overwhelmingly high weight [-> nearest neighbor interpolation]

Compute inverse distance interpolated values using `gstat::idw`

In [ ]:
library(gstat)
i2 <- idw(zinc~1, meuse.sf, meuse.gridStars, idp = 2)

In [ ]:
i025 <- idw(zinc~1, meuse.sf, meuse.gridStars, idp = 0.25)
p0.25 <- tm_shape(i025) + tm_raster(col = "var1.pred", style = "fisher", title = "Zinc (predicted)") +
  tm_shape(meuse.sf) + 
  tm_bubbles(size = "zinc", alpha = 0.2) +
  tm_layout(main.title = "p = 0.25")

p2 <- tm_shape(i2) + tm_raster(col = "var1.pred", style = "fisher", title = "Zinc (predicted)") +
  tm_shape(meuse.sf) + 
  tm_bubbles(size = "zinc", alpha = 0.2) +
  tm_layout(main.title = "p = 2")

i16 <- idw(zinc~1, meuse.sf, meuse.gridStars, idp = 16)
p16 <- tm_shape(i16) + tm_raster(col = "var1.pred", style = "fisher", title = "Zinc (predicted)") +
  tm_shape(meuse.sf) + 
  tm_bubbles(size = "zinc", alpha = 0.2) +
  tm_layout(main.title = "p = 16")

tmap_arrange(p0.25, p2, p16, nrow = 1)

### Kriging Interpolation Weights
In Kriging, the weight is a particular function of distance known as the **variogram** (or *semivariogram*)

Methods for modeling and analyzing spatial data rely heavily on stochastic properties of the spatial random field

- If what we observe must be considered a sample size of one, how can we learn anything about the variation of $Z(s)$ or the covariances among $Z(s_i)$ and $Z(s_j)$?
- Certain assumptions, such as having a random sample, provide the underpinnings of the stochastic structure that enables analysis

#### Random Field Properties: Stationarity 

**Stationarity** means that the random field looks similar in different parts of the domain $D$, it replicates itself

Consider two observations at $Z(s)$ and $Z(s+h)$

$h$ is a displacement by which we move from location $s$ to location $u=s+h$ (lag-vector)
	
- If random field is self-replicating, the stochastic properties of $Z(s)$ and $Z(s+h)$ should be similar
- To estimate the covariance between locations distance $h$ apart, we might consider all pairs $(Z(s),Z(s+h))$ in the estimation process, regardless of where $s_i$ is located 
- Stationarity is the absence of an origin – the spatial process has reached a state of equilibrium

***Time Series Example***

It doesn’t matter when a time shift is considered in term of absolute time, only how large the time shift is. We can talk about a difference of two days without worrying that the first occasion was a Saturday.

In the spatial context, stationarity means the lack of importance of absolute coordinates

The three important varieties of stationarity are *strict*, *second-order*, and *intrinsic*. We're going to focus on intrinsic stationarity, which assumes that the process that generated the
samples is a random function $Z(s)$ composed of a mean and residual
$$Z(s) = m + e(s)$$
with a constant mean
$$E(Z(s)) = m$$
and a variogram defined as
$$\gamma(h) = \frac{1}{2}E(Z(s) - Z(s + h))^2$$

Under the intrinsic stationarity assumption:

- Variance of $Z$ is constant
- Spatial correlation of $Z$ does not depend on location $s$, but only on
separation distance $h$ 

We can form multiple pairs ${z(s_i), z(s_j)}$ that
have (nearly) identical separation vectors $h = s_i - s_j$ and estimate correlation
from them

#### Random Field Properties: Isotropy

Stationarity reflects the lack of importance of absolute coordinates, but the direction in which the lag vector $h$ is assessed still plays an important role.

<img src="images/isotropy.jpg" width=600 />

The condition by which the random field is also invariant under rotation and reflection is known as **isotropy**.

- The covariance between any two points $Z(s)$ and $Z(s+h)$ is only a function of the Euclidean distance $||h||$ between the two points:  $Cov[Z(s),Z(s+h)]=C(||h||)$

- Under the isotropic assumption, the variogram can be estimated from $N_h$ sample
data pairs $Z(s_i)$, $Z(s_i + h)$ for a number of distances (or distance intervals)
$\tilde{h_j}$ by
$$\hat{\gamma}(\tilde{h_j})=\frac{1}{2N_h}\sum_{i=1}^{N_h}(Z(s_i)-Z(s_i+h))^2, \forall{h} \in \tilde{h_j}$$
and this estimate is called the *sample variogram*.

#### Variogram

It is because of the factor $\frac{1}{2}$ that $\gamma(h)$ is termed the semivariogram and $2\gamma(h)$ is termed the variogram 

The semivariogram is a structural tool that can convey a great deal about the nature and structure of spatial dependency in a random field

Estimating is a two-step process: 

1. Derive an empirical estimate of the semivariogram from the data
2. Fit a theoretical semivariogram model to the empirical estimate

- The semivariogram of a spatial process is one half of the variance of the difference between observations

- The semivariogram conveys information about the spatial structure and the degree of continuity of a random field

- Theoretical models are fit to the data to arrive at an estimated semivariogram that satisfies the properties needed for subsequent analysis

#### Exploratory Variogram Analysis

Visualize potential spatial correlation with an *h*-scatterplot (`gstat::hscat`):
a scatterplot of $Z(s_i)$ and $Z(s_j)$ pairs grouped according to their separation 
distance $h_{ij} = ||s_i - s_j||$

In [ ]:
hscat(zinc ~ 1, data = meuse.sf, breaks = (0:9) * 100)

- The diagonal lines represent perfect correlation
- It appears that nearer observations are more like one another, and that the
similarity declines with distance

Calculate the experimental variogram with the `gstat:: variogram` function. 

In [ ]:
v <- variogram(log(zinc) ~ 1, meuse.sf)

- `~ 1` defines a single constant predictor, leading to a spatially constant
mean coefficient

Explore spatial correlation by plotting the variogram cloud and related experimental variogram

- **Variogram cloud**: obtained by plotting all possible squared differences 
of observation pairs $(Z(s_i) - Z(s_j))^2$ against their separation distance $h_{ij}$ (`cloud = T` argument)
- **Experimental variogram**: plot of averages of the semivariogram cloud values over distance
intervals $h$

In [ ]:
cloudV <- plot(variogram(log(zinc) ~ 1, meuse.sf, cloud = TRUE), xlab = "distance h [m]",
     ylab = expression(gamma(h)), xlim = c(0, 1.055 * max(v$dist)), 
     main = "Variogram Cloud")

experV <- plot(variogram(log(zinc) ~ 1, meuse.sf), xlab = "distance h [m]",
     ylab = expression(gamma(h)), xlim = c(0, 1.055 * max(v$dist)), 
     main = "Experimental Variogram")

In [ ]:
library(gridExtra)
grid.arrange(cloudV, experV, nrow = 1)

A valid semivariogram has similar properties and conditions:

- *Range*: Distance at which maximum variance is reached
    - The distance at which data are no longer correlated, depending on scale it might not be present in all datasets
    - The *practical range* is the lag distance at which the semivariogram achieves 95% of the sill
    - Spatial autocorrelation exists only for pairs of points separated by less than the (practical) range
- *Sill*: Level of maximum variability within the dataset
    - $\lim_{x \to \infty} \gamma(h)$ approximates the variance within the dataset
    - The more quickly the semivariogram rises from the origin to the sill the more quickly autocorrelation declines
- *Nugget*: Variance at zero distance
    - Represents small scale variations or measurement error

<img src="images/variogram-example.jpg" width=600 />

#### Fitting Variogram Models

To progress toward spatial predictions, we need a variogram model $\gamma(h)$ for 
(potentially) all distances $h$, rather than the set of estimates

- Connecting estimates with straight lines, or assuming they reflect constant 
values over their respective distance intervals, would lead to statistical models 
with non-positive definite covariance matrices
- Couldn't use estimates in prediction

The traditional way of finding a suitable (semi)variogram model is to fit a parametric
model $\gamma(h; \theta)$ to the experimental variogram estimates $\hat\gamma(h_i)$.

- $\theta$ is a vector of parameters that is estimated from the data by direct or indirect methods
- In the isotropic models that follow, $\theta_0$ denotes the nugget and $\theta_s$ denotes the sill

#### Variogram Modeling: Common Models

- Nugget-Only Model:
    - The semivariogram of a white-noise problem
	  - The $Z(s_i)$ behave like a random sample, all having the same mean, variance with no correlations among them
	  - The model is devoid of spatial structure, the relative structured variability is zero
	  - Appropriate if the smallest sample distance in the data is greater than the range of the spatial process
	  
$$\gamma(h;\theta) = 
\Bigg\{\begin{array}
{ll}
0 & h=0 \\
\theta_s & h \neq 0
\end{array}$$

- Linear Model:
    - Intrinsically stationary with parameters $\theta_0$ and $\beta$ (both of which must be positive)
    - Covariances or semivariances usually don’t change linearly over a large range, but linear change of the semivariogram near the origin is often reasonable

$$\gamma(h;\theta) = 
\Bigg\{\begin{array}
{ll}
0 & h=0 \\
\theta_s + \beta ||h|| & h \neq 0
\end{array}$$

- Spherical Model:
    - One of the most popular semivariogram models in applied spatial statistics for second-order stationary random fields
  	- Linear behavior near the origin
	  - At a distance $\alpha$ the semivariogram meets the sill and remails flat thereafter
	  - Sometimes creates a kink at $||h||=\alpha$
	  
$$\gamma(h;\theta) = 
\Bigg\{\begin{array}
{lll}
0 & ||h||=0 \\
\theta_0 + \theta_s\{\frac{3}{2} \frac{||h||}{\alpha} - \frac{1}{2} (\frac{||h||}{\alpha})^3 \} & 0 < ||h|| \le \alpha \\
\theta_0 + \theta_s & ||h||>\alpha
\end{array}$$

- Exponential Model:
    – Found to fit spatial data in varied applications well
	  - Approaches the sill $\theta_s$ asymptotically as $||h|| \to \infty$
	  - In the parameterization shown below the parameter $\alpha$ is the practical range of the semivariogram
	  - For the same range and sill as the spherical model the exponential model rises more quickly from the origin and yields autocorrelations at short lag distances smaller than those of the spherical model
	  
$$\gamma(h;\theta) = 
\Bigg\{\begin{array}
{ll}
0 & h=0 \\
\theta_0 + \theta_s\Big\{1-exp \Big\{- \frac{3 ||h||}{\alpha} \Big\} \Big\} & h \neq 0
\end{array}$$

- Gaussian Model:
    - Found to fit spatial data in varied applications well
	  - Model exhibits quadratic behavior near the origin and produces short-range correlations that are higher for any of the other second-order stationary models with the same (practical) range
	  - The only difference between the Gaussian and exponential semivariogram is the square in the exponent
	  - Gaussian model is the most continuous near the origin of the models considered here – infinitely differentiable

$$\gamma(h;\theta) = 
\Bigg\{\begin{array}
{ll}
0 & h=0 \\
\theta_0 + \theta_s\Big\{1-exp \Big\{- 3 (\frac{||h||}{\alpha}) ^ 2 \Big\} \Big\} & h \neq 0
\end{array}$$

<img src="images/variogram-models2.jpg" width=600 />

#### Variogram Modeling

An overview of the basic variogram
models available in gstat is obtained by `show.vgms()`

In [ ]:
show.vgms(layout=c(4, 5))

In `gstat, valid variogram models are constructed by using one or combinations
of two or more basic variogram models

Variogram models are derived from data.frame objects, and are built by specifying the sill, model type and range:

In [ ]:
vgm(1, "Sph", 300)

Not all of these models are equally useful, in practice. 

Most practical studies have so far used exponential, spherical, Gaussian, or power models,
with or without a nugget, or a combination of those.

Can fit a variogram model using a variety of methods (e.g., least squares, maximum lielihood, composite likelihood). 

For weighted least squares fitting a variogram model to the experimental variogram, we need to take several steps:

1. Choose a suitable model (e.g., exponential, spherical), with or without
nugget
2. Choose suitable initial values for partial sill(s), range(s), and possibly
nugget
3. Fit this model, using one of the fitting criteria.

For our example variogram `v` on zinc deposits, the spherical model looks like a 
reasonable choice

Initial values for the variogram fit are needed for `fit.variogram`, because
for the spherical model (and many other models) fitting the range parameter
involves non-linear regression. 

The following fit works:

In [ ]:
v.fit <- fit.variogram(v, vgm(1, "Sph", 800, 1))

plot(v, v.fit, plot.numbers = TRUE, xlab = "distance h [m]",
     ylab = expression(gamma(h)),
     xlim = c(0, 1.055 * max(v$dist)))

<a id="Ex2"></a>
### **<span style='color:Green'> Exercise 2: Variogram (15 minutes)</span>**   

1. Create an experimental variogram object for ordinary kriging weights using the 
formula `log(Iron) ~ 1` (`v`) and then fit a variogram model (`v.fit`) to `v`. 

    When fitting your variogram model you need to think about the sill, model type 
    (e.g., "Exp", "Sph", "Gau", "Mat"), range, and nugget values. See `fit.variogram` for
    additional arguments depending on the model chosen. It doesn't matter which model you 
    choose, but you do need to defend your choice. 

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
library(gstat)

v <- variogram(log(Iron) ~ 1, mesabiRngWells)
v.fit <- fit.variogram(v, vgm(psill = 4, model = "Exp", range = 20000, nugget = 4))
```
----
          
</details>

2. Create an experimental variogram object for universal kriging weights using the 
formula `log(Iron) ~ elevation` (`vt`) and then fit a variogram model (`vt.fit`) to `vt`. 

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
vt <- variogram(log(Iron) ~ elevation, mesabiRngWells)
vt.fit <- fit.variogram(vt, vgm(psill = 4, model = "Exp", range = 20000, nugget = 4))
```
----
          
</details>

3. Plot your experimental and fitted variograms for both 2.1 and 2.2. An example answer is shown below, but note that my choice of variogram models may differ from yours.

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
plot(v, v.fit, plot.numbers = TRUE, xlab = "distance h [m]",
     ylab = expression(gamma(h)), 
     main = "Experimental and Fitted \n Variograms Without Covariates", 
     main.title.size = 0.75)

plot(vt, vt.fit, plot.numbers = TRUE, xlab = "distance h [m]",
     ylab = expression(gamma(h)), 
     main = "Experimental and Fitted \n Variograms With Covariates", 
     main.title.size = 075)
```
----
          
</details>

<a id="Kriging"></a>
## Kriging

Spatial prediction refers to the prediction of unknown quantities $Z(s_0)$, based
on sample data $Z(s_i)$ and assumptions regarding the form of the trend of $Z$
and its variance and spatial correlation

Kriging methods were developed to predict $Z(s_0)$ at some unsampled location $s_0$ based on the sample $Z(s_1),…,Z(s_n)$

The instances of this best linear unbiased prediction method with the number
of predictors $p > 0$ are usually called *universal kriging*.

When $p = 0$ and $X_0 \equiv 1$, the corresponding
prediction is called *ordinary kriging*.

*Simple kriging* is obtained when $\beta$ is a priori assumed
to be known. 

We can krige `zinc` by using `gstat::krige`, with the model for the trend, the data, 
the prediction grid, and the variogram model as arguments

Simple Kriging

In [ ]:
lz.sk <- krige(log(zinc) ~ 1, meuse.sf, meuse.gridStars, v.fit, beta = 5.9)

Ordinary Kriging

In [ ]:
lz.ok <- krige(log(zinc) ~ 1, meuse.sf, meuse.gridStars, v.fit)

Universal Kriging

In [ ]:
vt <- variogram(log(zinc) ~ sqrt(dist), meuse.sf)
vt.fit <- fit.variogram(vt, vgm(1, "Exp", 300, 1))
lz.uk <- krige(log(zinc) ~ sqrt(dist), meuse.sf, meuse.gridStars, vt.fit)

`krige` chooses the kriging method itself, depending on
the information it is provided with: are trend coefficients given? is the trend
constant or more complex?

In [ ]:
sk <- tm_shape(lz.sk) + tm_raster(col = "var1.pred", style = "fisher", title = "Zinc (predicted)") +
  tm_shape(meuse.sf) + 
  tm_bubbles(size = "zinc", alpha = 0.2) +
  tm_layout(main.title = "Simple Kriging")

ok <- tm_shape(lz.ok) + tm_raster(col = "var1.pred", style = "fisher", title = "Zinc (predicted)") +
  tm_shape(meuse.sf) + 
  tm_bubbles(size = "zinc", alpha = 0.2) +
  tm_layout(main.title = "Ordinary Kriging")

uk <- tm_shape(lz.uk) + tm_raster(col = "var1.pred", style = "fisher", title = "Zinc (predicted)") +
  tm_shape(meuse.sf) + 
  tm_bubbles(size = "zinc", alpha = 0.2) +
  tm_layout(main.title = "Universal Kriging")

tmap_arrange(sk, ok, uk, nrow = 1)

<a id="Ex3"></a>
### **<span style='color:Green'> Exercise 3: Prediction with Manual Variogram (10 minutes)</span>**   

Predict the levels of iron in groundwater throughout the Mesabi Range using

1. Inverse Distance Weighting (`p = 2`)
2. Ordinary Kriging
3. Universal Kriging (covariate = elevation)

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
idw <- idw(Iron~1, 
           mesabiRngWells, 
           mesabiRngDEM, idp = 2)

k.ok <- krige(log(Iron) ~ 1, 
              mesabiRngWells, 
              mesabiRngDEM, v.fit)

k.uk <- krige(formula = log(Iron) ~ elevation, 
              mesabiRngWells, 
              mesabiRngDEM, 
              model = vt.fit)
```
----
          
</details>

Create a side-by-side figure of your three maps and compare. An example answer is
    shown below, but note that my choice of underlying variogram models may differ from yours.

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
idwMap = tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(idw) + tm_raster(col = "var1.pred", style = "fisher", 
                            title = "Iron (predicted)") +
  tm_shape(mesabiRngWells) + tm_bubbles(size = "Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "IDW Predicted Iron Levels in the \n Mesabi Range", 
            main.title.size = 0.75, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

okMap <- tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(k.ok) + tm_raster(col = "var1.pred", style = "fisher", 
                             title = "Iron (predicted)") +
  tm_shape(mesabiRngWells) + tm_bubbles(size = "Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "Ordinary Kriging Predicted Iron \n Levels in the Mesabi Range", 
            main.title.size = 0.75, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

ukMap <- tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(k.uk) + tm_raster(col = "var1.pred", style = "fisher", 
                             title = "Iron (predicted)") +
  tm_shape(mesabiRngWells) + tm_bubbles(size = "Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "Universal Kriging Predicted Iron \n Levels in the Mesabi Range", 
            main.title.size = 0.75, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

tmap_arrange(idwMap, okMap, ukMap, nrow = 1)
```
----
          
</details>

<a id="Autofit"></a>
## Automatically Fitting Variogram

It is not easy to manually fit a variogram model, which is why the `automap` package
was created by Paul Hiemstra!

This package automatically fits a variogram to the data 

Fitting is done through the `fit.variogram` function

- Normally, in `fit.variogram` user supplies an initial estimate for the sill, range etc. 
- `automap::autofitVariogram` provides this estimate based on the data and then calls `fit.variogram`

<a id="Ex4"></a>
### **<span style='color:Green'> Exercise 4: Prediction with Automatic Variogram (5 minutes)</span>**   

1. Using `autofitVariogram` automatically fit your experimental variogram objects for
    ordinary (name this `v.fitA`) and universal (name this `vt.fitA`) kriging. 
    
2. Plot your autofit variograms using a simple `plot` command. Compare your autofitted
    variograms to your manually fitted variograms.

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
library(automap)

v.fitA <- autofitVariogram(log(Iron) ~ 1, as(mesabiRngWells, "Spatial"))
vt.fitA <- autofitVariogram(log(Iron) ~ elevation, as(mesabiRngWells, "Spatial"))

vModA <- plot(v.fitA)
vtModA <- plot(vt.fitA)
```
----
          
</details>

3. Again, predict the levels of iron in groundwater throughout the Mesabi Range using

   1. Inverse Distance Weighting (`p = 2`)
   2. Ordinary Kriging
   3. Universal Kriging (covariate = elevation)

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
idwA <- idw(Iron~1, 
            mesabiRngWells, 
            mesabiRngDEM, idp = 2)

k.okA <- krige(log(Iron) ~ 1, 
               mesabiRngWells, 
               mesabiRngDEM, 
               model = v.fitA$var_model)

k.ukA <- krige(formula = log(Iron) ~ elevation, 
               mesabiRngWells, 
               mesabiRngDEM, 
               model = vt.fitA$var_model)

```
----
          
</details>

Now use your automatically fitted variograms [Note: you can call the 
    variogram model from your fitted object using `v.fitA$var_model` or `vt.fitA$var_model`]. 
    Create a side-by-side figure of your three maps and compare to the maps from exercise 3.

<details>
    
<summary><span style='color:Green'> Click to see answer  </span></summary>

```r
idwMapA = tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(idwA) + tm_raster(col = "var1.pred", style = "fisher", 
                             title = "Iron (predicted)") +
  tm_shape(mesabiRngWells) + tm_bubbles(size = "Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "IDW Predicted Iron Levels in the \n Mesabi Range", 
            main.title.size = 0.75, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

okMapA <- tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(k.okA) + tm_raster(col = "var1.pred", style = "fisher", 
                              title = "Iron (predicted)") +
  tm_shape(mesabiRngWells) + tm_bubbles(size = "Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "Ordinary Kriging Predicted Iron Levels \n in the Mesabi Range", 
            main.title.size = 0.75, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

ukMapA <- tm_shape(mesabiRngCnty) + tm_borders() +
  tm_shape(mn_adm2) + tm_polygons() +
  tm_shape(lakeSup) + tm_polygons("skyblue2") +
  tm_shape(k.ukA) + tm_raster(col = "var1.pred", style = "fisher", 
                              title = "Iron (predicted)") +
  tm_shape(mesabiRngWells) + tm_bubbles(size = "Iron", alpha = 0.2) +
  tm_shape(mesabiRngCnty) + tm_borders() +
  tm_layout(main.title = "Universal Kriging Predicted Iron Levels \n in the Mesabi Range", 
            main.title.size = 0.75, 
            legend.position = c("left","top"), 
            legend.bg.color = "white")

tmap_arrange(idwMapA, okMapA, ukMapA, nrow = 1)
```
----
          
</details>